# buy-open-sell-close

On the first trading day of each week, buy on the open, then sell on the close.

In [1]:
import pandas as pd
import datetime
import pinkfish as pf

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [2]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.now()

Timeseries

In [3]:
# Fetch timeseries, select.
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)

# Add calendar columns.
ts = pf.calendar(ts)

# Finalize timeseries.
ts, start = pf.finalize_timeseries(ts, start)

# Create tradelog and daily balance objects.
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

In [4]:
ts

,open,high,low,close,adj_close,volume,dotw,dotm,doty,month,first_dotw,first_dotm,first_doty,last_dotw,last_dotm,last_doty
date,,,,,,,,,,,,,,,,
2015-01-02,206.38,206.88,204.18,205.43,174.81,121465900,4,2,2,1,False,True,True,True,False,False
2015-01-05,204.17,204.37,201.35,201.72,171.65,169632600,0,5,5,1,True,False,False,False,False,False
2015-01-06,202.09,202.72,198.86,199.82,170.03,209151400,1,6,6,1,False,False,False,False,False,False
2015-01-07,201.42,202.72,200.88,202.31,172.15,125346700,2,7,7,1,False,False,False,False,False,False
2015-01-08,204.01,206.16,203.99,205.90,175.21,147217800,3,8,8,1,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-15,469.49,470.70,467.43,469.33,469.33,141319300,4,15,349,12,False,False,False,True,False,False
2023-12-18,470.98,472.98,469.89,471.97,471.97,70375300,0,18,352,12,True,False,False,False,False,False
2023-12-19,472.53,474.92,472.45,474.84,474.84,55761800,1,19,353,12,False,False,False,False,False,False


Algorithm

In [5]:
pf.TradeLog.cash = capital

# Loop through timeseries.
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # On first trading day of the week, buy open, then sell close.
    if row.first_dotw:
        tlog.buy(date, row.open)
        tlog.sell(date, row.close)

    # Record daily balance.
    dbal.append(date, row.close, row.high, row.low)

Retrieve logs

In [6]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [7]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
463,2023-11-20,450.53,2023-11-20,454.26,3.73,93.25,25,1559.61,LONG,SPY
464,2023-11-27,454.65,2023-11-27,454.48,-0.17,-4.25,25,1555.36,LONG,SPY
465,2023-12-04,455.60,2023-12-04,456.69,1.09,27.25,25,1582.61,LONG,SPY
466,2023-12-11,459.69,2023-12-11,461.99,2.30,57.50,25,1640.11,LONG,SPY
467,2023-12-18,470.98,2023-12-18,471.97,0.99,23.76,24,1663.87,LONG,SPY


Get stats

In [8]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [9]:
pf.summary(stats)

,strategy
annual_return_rate,1.73
max_closed_out_drawdown,-13.98
best_month,6.04
worst_month,-5.08
sharpe_ratio,0.35
sortino_ratio,0.21
monthly_std,1.41
annual_std,5.88
